<a href="https://colab.research.google.com/github/davidofitaly/notes_03_python_in_data_analysis/blob/main/07_data_aggregation_and_group_operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

##7.1 Mechanics of the groupby interface

###Groupby

In [28]:
# Create data
data = {
    "key_1": np.random.choice(list("abc"), size=10),  # Random letters from 'a' to 'd'
    "key_2": np.random.choice([0, 1], size=10),  # Random binary values (0 or 1)
    "data_1": np.random.normal(loc=0, scale=1, size=10),  # Normally distributed data (mean=0, std=1)
    "data_2": np.random.normal(loc=5, scale=2, size=10)  # Normally distributed data (mean=5, std=2)
}

# Create DataFrame
df = pd.DataFrame(data)

df

,key_1,key_2,data_1,data_2
0,a,1,-1.039777,0.123997
1,a,1,0.616202,5.642068
2,c,1,2.747263,7.230746
3,b,1,0.664580,5.955566
4,c,0,-2.041287,6.648567
5,b,1,0.332344,9.001822
6,b,0,1.500993,1.958932
7,b,1,1.034211,2.896610
8,c,1,0.159162,6.429005
9,a,0,-0.042617,3.588579




*   


In [39]:
df['data_1'].groupby(df['key_1']).mean()

,data_1
key_1,
a,-0.155397
b,0.883032
c,0.288379




*  



In [30]:
df['data_1'].groupby([df['key_1'], df['key_2']]).mean()

key_1  key_2
a      0       -0.042617
       1       -0.211787
b      0        1.500993
       1        0.677045
c      0       -2.041287
       1        1.453213
Name: data_1, dtype: float64



*   


In [33]:
df.groupby("key_1").mean()

,key_2,data_1,data_2
key_1,,,
a,0.666667,-0.155397,3.118214
b,0.750000,0.883032,4.953233
c,0.666667,0.288379,6.769439




*   



In [35]:
df.groupby("key_2").mean(numeric_only=True)

,data_1,data_2
key_2,,
0,-0.194304,4.065359
1,0.644855,5.325688




*  


In [36]:
df.groupby(["key_1", "key_2"]).mean()

data_1    data_2
key_1 key_2                    
a     0     -0.042617  3.588579
      1     -0.211787  2.883032
b     0      1.500993  1.958932
      1      0.677045  5.951333
c     0     -2.041287  6.648567
      1      1.453213  6.829876



*  



In [37]:
df.groupby(["key_1", "key_2"]).size()

key_1  key_2
a      0        1
       1        2
b      0        1
       1        3
c      0        1
       1        2
dtype: int64



*   



In [38]:
df.groupby(["key_1", "key_2"]).count()

data_1  data_2
key_1 key_2                
a     0           1       1
      1           2       2
b     0           1       1
      1           3       3
c     0           1       1
      1           2       2

###Iteration through groups



*   



In [44]:
for name, group in df.groupby("key_1"):
    print(f"{name.upper()}")
    print(group)

A
  key_1  key_2    data_1    data_2
0     a      1 -1.039777  0.123997
1     a      1  0.616202  5.642068
9     a      0 -0.042617  3.588579
B
  key_1  key_2    data_1    data_2
3     b      1  0.664580  5.955566
5     b      1  0.332344  9.001822
6     b      0  1.500993  1.958932
7     b      1  1.034211  2.896610
C
  key_1  key_2    data_1    data_2
2     c      1  2.747263  7.230746
4     c      0 -2.041287  6.648567
8     c      1  0.159162  6.429005




*  


In [54]:
for (k1, k2), group in df.groupby(['key_1', 'key_2']):
    print((k1, k2))
    print(group, end='\n\n')

('a', 0)
  key_1  key_2    data_1    data_2
9     a      0 -0.042617  3.588579

('a', 1)
  key_1  key_2    data_1    data_2
0     a      1 -1.039777  0.123997
1     a      1  0.616202  5.642068

('b', 0)
  key_1  key_2    data_1    data_2
6     b      0  1.500993  1.958932

('b', 1)
  key_1  key_2    data_1    data_2
3     b      1  0.664580  5.955566
5     b      1  0.332344  9.001822
7     b      1  1.034211  2.896610

('c', 0)
  key_1  key_2    data_1    data_2
4     c      0 -2.041287  6.648567

('c', 1)
  key_1  key_2    data_1    data_2
2     c      1  2.747263  7.230746
8     c      1  0.159162  6.429005



###Selecting a column or a subset of columns



*   



In [55]:
df.groupby(['key_1', 'key_2'])[['data_2']].mean()

data_2
key_1 key_2          
a     0      3.588579
      1      2.883032
b     0      1.958932
      1      5.951333
c     0      6.648567
      1      6.829876

##7.2 The apply method

###Apply

In [100]:
# Create DataFrame with names as index and letters + numbers as columns
names = ["Alice", "Bob", "Charlie", "David", "Eve"]
columns = list("abc23")
data2 = np.random.randn(5, 5)
data2[np.random.rand(5, 5) < 0.2] = np.nan  # Introduce some NaN values

df_2 = pd.DataFrame(data2, index=names, columns=columns)

df_2

,a,b,c,2,3
Alice,0.537381,-1.256041,0.639771,-1.260158,1.506871
Bob,0.397567,0.281626,-1.207256,NaN,-1.754887
Charlie,-0.298032,0.194427,1.413695,-0.626073,-1.510192
David,NaN,-0.669861,-0.047127,NaN,NaN
Eve,-0.244824,0.731670,-0.570766,-2.728289,0.045978




*



In [97]:
df_2.apply(lambda x: x.mean(), axis=1)

,0
Alice,0.200591
Bob,-0.550546
Charlie,-0.540683
David,-0.414475
Eve,0.977867




*  



In [98]:
df_2.apply(lambda x: x.sum(), axis=1)

,0
Alice,0.802365
Bob,-2.752732
Charlie,-2.162731
David,-2.072377
Eve,1.955733




*   





In [102]:
df_2.apply(lambda x: x.notna().sum(), axis=1)

,0
Alice,5
Bob,4
Charlie,5
David,2
Eve,5




*  


In [103]:
df_2.apply(lambda x: x.max() - x.min(), axis=1)

,0
Alice,2.767030
Bob,2.152455
Charlie,2.923887
David,0.622734
Eve,3.459959




*   


In [104]:
df_2.apply(lambda x: (x - x.mean()) / x.std(), axis=1)

,a,b,c,2,3
Alice,0.407021,-1.041840,0.489739,-1.045166,1.190247
Bob,0.900148,0.792367,-0.591715,NaN,-1.100800
Charlie,-0.123017,0.333174,1.462648,-0.426899,-1.245906
David,NaN,-0.707107,0.707107,NaN,NaN
Eve,0.235920,0.982864,-0.013401,-1.663744,0.458361




*   



In [105]:
df_2.apply(lambda x: (x > 0).sum(), axis=1)

,0
Alice,3
Bob,2
Charlie,2
David,0
Eve,2




*   



In [106]:
df_2.apply(lambda x: (x.max() - x.min())**2, axis=1)

,0
Alice,7.656454
Bob,4.633061
Charlie,8.549116
David,0.387798
Eve,11.971316


###Quantiles and basket analysis

In [114]:
# Creating a DataFrame with two columns 'data_1' and 'data_2' from a normal distribution
np.random.seed(42)  # Set seed for reproducibility
data_1 = np.random.normal(0, 1, 1000)  # 1000 samples from a normal distribution
data_2 = np.random.normal(5, 2, 1000)  # 1000 samples from a normal distribution

df_3 = pd.DataFrame({
    'data_1': data_1,
    'data_2': data_2
})

df_3

,data_1,data_2
0,0.496714,7.798711
1,-0.138264,6.849267
2,0.647689,5.119261
3,1.523030,3.706126
4,-0.234153,6.396447
...,...,...
995,-0.281100,7.140300
996,1.797687,4.946957
997,0.640843,3.236251
998,-0.571179,4.673866




*   


In [116]:
df_3['data_1_cut'] = pd.cut(df['data_1'], bins=5)
df_3.head()  # Show the first 5 rows

,data_1,data_2,data_1_cut
0,0.496714,7.798711,"(-0.404, 1.015]"
1,-0.138264,6.849267,"(-0.404, 1.015]"
2,0.647689,5.119261,"(-0.404, 1.015]"
3,1.523030,3.706126,"(1.015, 2.434]"
4,-0.234153,6.396447,"(-0.404, 1.015]"




*   


In [117]:
df_3['data_2_qcut'] = pd.qcut(df['data_2'], q=4)
df_3.head()  # Show the first 5 rows

,data_1,data_2,data_1_cut,data_2_qcut
0,0.496714,7.798711,"(-0.404, 1.015]","(6.458, 11.386]"
1,-0.138264,6.849267,"(-0.404, 1.015]","(6.458, 11.386]"
2,0.647689,5.119261,"(-0.404, 1.015]","(3.788, 5.126]"
3,1.523030,3.706126,"(1.015, 2.434]","(-0.882, 3.788]"
4,-0.234153,6.396447,"(-0.404, 1.015]","(5.126, 6.458]"




*   


In [118]:
df_3['data_1_cut'].value_counts()

,count
data_1_cut,
"(-0.404, 1.015]",498
"(-1.822, -0.404]",324
"(1.015, 2.434]",142
"(-3.248, -1.822]",26
"(2.434, 3.853]",10




*  


In [119]:
df_3.groupby('data_1_cut')[['data_1', 'data_2']].mean()

<ipython-input-119-a67cceab73fb>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_3.groupby('data_1_cut')[['data_1', 'data_2']].mean()


,data_1,data_2
data_1_cut,,
"(-3.248, -1.822]",-2.173346,5.181929
"(-1.822, -0.404]",-0.918304,5.263872
"(-0.404, 1.015]",0.270195,5.117978
"(1.015, 2.434]",1.489463,4.920950
"(2.434, 3.853]",2.730883,5.391990


##7.3 Pivot tables and cross tables

In [121]:
data = {
    'Category': np.random.choice(['A', 'B', 'C'], size=1000),
    'Subcategory': np.random.choice(['X', 'Y', 'Z'], size=1000),
    'Value': np.random.randn(1000) * 100,
    'Date': pd.date_range('2022-01-01', periods=1000, freq='D')
}

df = pd.DataFrame(data)
df.head()

,Category,Subcategory,Value,Date
0,B,X,137.040969,2022-01-01
1,C,X,142.945780,2022-01-02
2,B,Z,146.445431,2022-01-03
3,C,X,-56.158794,2022-01-04
4,C,Z,198.985507,2022-01-05




*


In [123]:
df.pivot_table(values='Value', index='Category', columns='Subcategory', aggfunc='mean')


Subcategory,X,Y,Z
Category,,,
A,0.843252,-6.092376,2.282928
B,-1.716918,-3.046588,12.724579
C,13.476721,-4.142124,5.192808




*  



In [124]:
df.pivot_table(values='Value', index='Category', columns='Subcategory',
                                    aggfunc=['mean', 'sum'])

mean                               sum              \
Subcategory          X         Y          Z            X           Y   
Category                                                               
A             0.843252 -6.092376   2.282928    93.600962 -688.438453   
B            -1.716918 -3.046588  12.724579  -199.162440 -335.124689   
C            13.476721 -4.142124   5.192808  1307.241917 -430.780856   

                          
Subcategory            Z  
Category                  
A             292.214828  
B            1234.284196  
C             643.908177



*   




In [125]:
pd.crosstab(df['Category'], df['Subcategory'])

Subcategory,X,Y,Z
Category,,,
A,111,113,128
B,116,110,97
C,97,104,124




*   


In [126]:
pd.crosstab(df['Category'], df['Subcategory'], normalize='all')

Subcategory,X,Y,Z
Category,,,
A,0.111,0.113,0.128
B,0.116,0.110,0.097
C,0.097,0.104,0.124
